<a href="https://colab.research.google.com/github/Maryam-Mostafa/Pyspark-/blob/master/PySpark%20with%20SQL%20and%20RandomForest%20model.ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 2 PySpark:

### Machine Learning

#### But first, let's do some SQL :)

**First, install and import PySPark and SparkSession**

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 31 kB/s 
     |████████████████████████████████| 198 kB 42.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=ba67d8afb2127fcfd8b4d520de62fccad527dc26aae862ed205983cb7f1570c7
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]") \
                    .appName('SparkByExamples.com') \
                    .getOrCreate()

print('PySpark Version :'+spark.version)

PySpark Version :3.2.1


**Now, download the dataset**

In [ ]:
!gdown https://drive.google.com/uc?id=1PB6wBDVTM_eocxOyi0lWlLBQOlH0rLe_ -O PatientInfo.csv

Downloading...
From: https://drive.google.com/uc?id=1PB6wBDVTM_eocxOyi0lWlLBQOlH0rLe_
To: /content/PatientInfo.csv
100% 489k/489k [00:00<00:00, 106MB/s]


**Create a SparkSession object and name the app "Lab2"**

In [ ]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('Lab2_G2').getOrCreate()

**1. Read the file PatientInfo.csv into a dataframe**

In [ ]:
df = spark.read.csv('/content/PatientInfo.csv', header = True, inferSchema=True)

**Show the first 5 lines of the dataframe**

In [ ]:
df.show(5)

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|         null|released|
|1000000002|  male|30s|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|  Jongno-gu|contact with patient| 2002000001|            17|              null|    2020-01-30|   202

**Now do the same but using SQL select statement**

1. Create a temporary view (table) called patients

In [ ]:
df.createOrReplaceTempView("patients")

2. Use SELECT statement to select all columns from the dataframe

In [ ]:
sqlDF = spark.sql("SELECT * FROM patients")
sqlDF.show()

+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|        city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul|  Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|         null|released|
|1000000002|  male|30s|  Korea|   Seoul| Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|   Jongno-gu|contact with patient| 2002000001|            17|              null|    2020-01-30|

3. Limit the output to only 5 rows *using SQL commands*

In [ ]:
sqlDF = spark.sql("SELECT * FROM patients limit 5")
sqlDF.show()

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|         null|released|
|1000000002|  male|30s|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|  Jongno-gu|contact with patient| 2002000001|            17|              null|    2020-01-30|   202

4. Select the count of males and females in the dataset

In [ ]:
sqlDF4 = spark.sql("select SUM(case when sex = 'male' then 1 end) as Males_count\
                  , SUM(case when sex = 'female' then 1 end) as Females_count FROM patients")
sqlDF4.show()

+-----------+-------------+
|Males_count|Females_count|
+-----------+-------------+
|       1825|         2218|
+-----------+-------------+



5. Select the count of males and females *as percentage* (how many percent of the data are males and how many are females?)

In [ ]:
sqlDF5 = spark.sql("select ((SUM(case when sex = 'male' then 1 end))/count(*))*100 as Males_percentage\
                  , (SUM(case when sex = 'female' then 1 end)/count(*))*100 as Females_percentage FROM patients")
sqlDF5.show()

+-----------------+------------------+
| Males_percentage|Females_percentage|
+-----------------+------------------+
|35.33397870280736|42.942884801548885|
+-----------------+------------------+



6. How many people did survive, and how many didn't?

In [ ]:
sqlDF6 = spark.sql("select SUM(case when state = 'deceased' then 1 end) as not_survived\
                  , SUM(case when state != 'deceased' then 1 end) as survived FROM patients")
sqlDF6.show()

+------------+--------+
|not_survived|survived|
+------------+--------+
|          78|    5087|
+------------+--------+



Now, let's perform some preprocessing using SQL:

1. Convert *age* column to double after removing the 's' at the end -- *hint: check SUBSTRING method*
2. Select only the following columns: `['sex', 'age', 'province', 'state']`
3. Store the result of the query in a new dataframe

In [ ]:
newDF = spark.sql("select sex, double(substring(age,1,length(age)-1)) as age, province, state from patients")
newDF.show()

+------+----+--------+--------+
|   sex| age|province|   state|
+------+----+--------+--------+
|  male|50.0|   Seoul|released|
|  male|30.0|   Seoul|released|
|  male|50.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|female|20.0|   Seoul|released|
|female|50.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|  male|30.0|   Seoul|released|
|female|60.0|   Seoul|released|
|female|50.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|  male|80.0|   Seoul|deceased|
|female|60.0|   Seoul|released|
|  male|70.0|   Seoul|released|
|  male|70.0|   Seoul|released|
|  male|70.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|female|70.0|   Seoul|released|
|female|70.0|   Seoul|released|
+------+----+--------+--------+
only showing top 20 rows



Now view the new dataframe to make sure everything is alright

In [ ]:
newDF.printSchema()

root
 |-- sex: string (nullable = true)
 |-- age: double (nullable = true)
 |-- province: string (nullable = true)
 |-- state: string (nullable = true)



**Now, let's get back to spark operations**

Please copy the following operations from your solution in Lab 1
___

Add a **is_dead** column if patient state is not released then it should yield true, else then False

In [ ]:
 from pyspark.sql.functions import when, lit

df1 = newDF.withColumn("is_dead", \
   when((newDF.state == 'deceased'), lit(1))\
     .otherwise(lit(0)) \
  )
df1.show()

+------+----+--------+--------+-------+
|   sex| age|province|   state|is_dead|
+------+----+--------+--------+-------+
|  male|50.0|   Seoul|released|      0|
|  male|30.0|   Seoul|released|      0|
|  male|50.0|   Seoul|released|      0|
|  male|20.0|   Seoul|released|      0|
|female|20.0|   Seoul|released|      0|
|female|50.0|   Seoul|released|      0|
|  male|20.0|   Seoul|released|      0|
|  male|20.0|   Seoul|released|      0|
|  male|30.0|   Seoul|released|      0|
|female|60.0|   Seoul|released|      0|
|female|50.0|   Seoul|released|      0|
|  male|20.0|   Seoul|released|      0|
|  male|80.0|   Seoul|deceased|      1|
|female|60.0|   Seoul|released|      0|
|  male|70.0|   Seoul|released|      0|
|  male|70.0|   Seoul|released|      0|
|  male|70.0|   Seoul|released|      0|
|  male|20.0|   Seoul|released|      0|
|female|70.0|   Seoul|released|      0|
|female|70.0|   Seoul|released|      0|
+------+----+--------+--------+-------+
only showing top 20 rows



**Please split the data into train and test dataframes**

*Ratio: 80:20 - Seed=42*

In [ ]:
train, test = df1.randomSplit([0.8,0.2], seed = 42)

In [ ]:
train.show(5)

+----+----+-----------+--------+-------+
| sex| age|   province|   state|is_dead|
+----+----+-----------+--------+-------+
|null|null|      Busan|isolated|      0|
|null|null|      Busan|isolated|      0|
|null|null|Gyeonggi-do|isolated|      0|
|null|null|Gyeonggi-do|isolated|      0|
|null|null|Gyeonggi-do|isolated|      0|
+----+----+-----------+--------+-------+
only showing top 5 rows



In [ ]:
test.show(5)

+----+----+-----------+--------+-------+
| sex| age|   province|   state|is_dead|
+----+----+-----------+--------+-------+
|null|null|Gyeonggi-do|isolated|      0|
|null|null|Gyeonggi-do|isolated|      0|
|null|null|Gyeonggi-do|isolated|      0|
|null|null|Gyeonggi-do|isolated|      0|
|null|null|Gyeonggi-do|isolated|      0|
+----+----+-----------+--------+-------+
only showing top 5 rows



**Now, let's import RandomForestClassifier and start our ML pipeline**

In [ ]:
from pyspark.mllib.tree import RandomForest

**Create a pipeline that contains the following stages:**

- Imputer: impute the null values in `age` column to the mean value
- StringIndexer: convert `sex` to `is_male` and `province` to `province_index` as numerical values
- OneHotEncoder: perform one hot encoding on both `is_male` and -province_index`
- VectorAssembler: assemble feature vector from the following columns: `'age', 'is_male', 'province_index'`
- RandomForestClassifier: final estimator

In [ ]:
from pyspark.ml.feature import Imputer

imputer = Imputer(inputCol="age", outputCol="age").setStrategy("mean")
model1 = imputer.fit(df1)
model1.transform(df1).show()

+------+----+--------+--------+-------+
|   sex| age|province|   state|is_dead|
+------+----+--------+--------+-------+
|  male|50.0|   Seoul|released|      0|
|  male|30.0|   Seoul|released|      0|
|  male|50.0|   Seoul|released|      0|
|  male|20.0|   Seoul|released|      0|
|female|20.0|   Seoul|released|      0|
|female|50.0|   Seoul|released|      0|
|  male|20.0|   Seoul|released|      0|
|  male|20.0|   Seoul|released|      0|
|  male|30.0|   Seoul|released|      0|
|female|60.0|   Seoul|released|      0|
|female|50.0|   Seoul|released|      0|
|  male|20.0|   Seoul|released|      0|
|  male|80.0|   Seoul|deceased|      1|
|female|60.0|   Seoul|released|      0|
|  male|70.0|   Seoul|released|      0|
|  male|70.0|   Seoul|released|      0|
|  male|70.0|   Seoul|released|      0|
|  male|20.0|   Seoul|released|      0|
|female|70.0|   Seoul|released|      0|
|female|70.0|   Seoul|released|      0|
+------+----+--------+--------+-------+
only showing top 20 rows



the pipeline structure 

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline

In [ ]:
stringIndexer = StringIndexer(inputCols=['sex','province'],outputCols=['is_male', 'province_index'], handleInvalid='skip')

oheEncoder = OneHotEncoder(inputCols=['is_male', 'province_index'], outputCols=['is_male_OHE', 'province_index_OHE'])

vecAssembler = VectorAssembler(inputCols=['age', 'is_male_OHE', 'province_index_OHE'],outputCol='features', handleInvalid = "skip")

In [ ]:
rf = RandomForestClassifier(featuresCol='features',labelCol='is_dead', numTrees=3, maxDepth=2, seed=42)

In [ ]:
stgs = [stringIndexer,oheEncoder,vecAssembler,rf]

In [ ]:
pl = Pipeline(stages=stgs)

Fit the pipeline to the train dataframe

In [ ]:
pl_Model = pl.fit(train)

Now transform the test DF to get predictions

In [ ]:
predDF = pl_Model.transform(test)

Show the final predictions DF

In [ ]:
predDF.show(5)

+------+----+-----------------+--------+-------+-------+--------------+-------------+------------------+--------------------+--------------------+--------------------+----------+
|   sex| age|         province|   state|is_dead|is_male|province_index|  is_male_OHE|province_index_OHE|            features|       rawPrediction|         probability|prediction|
+------+----+-----------------+--------+-------+-------+--------------+-------------+------------------+--------------------+--------------------+--------------------+----------+
|female| 0.0|      Gyeonggi-do|isolated|      0|    0.0|           2.0|(1,[0],[1.0])|    (16,[2],[1.0])|(18,[1,4],[1.0,1.0])|[2.95756849679989...|[0.98585616559996...|       0.0|
|female| 0.0|      Gyeonggi-do|isolated|      0|    0.0|           2.0|(1,[0],[1.0])|    (16,[2],[1.0])|(18,[1,4],[1.0,1.0])|[2.95756849679989...|[0.98585616559996...|       0.0|
|female| 0.0| Gyeongsangbuk-do|released|      0|    0.0|           1.0|(1,[0],[1.0])|    (16,[1],[1.0])|(

In [ ]:
predDF.select('features','is_dead','prediction').show(5,truncate=False)

+----------------------------+-------+----------+
|features                    |is_dead|prediction|
+----------------------------+-------+----------+
|(18,[1,4],[1.0,1.0])        |0      |0.0       |
|(18,[1,4],[1.0,1.0])        |0      |0.0       |
|(18,[1,3],[1.0,1.0])        |0      |0.0       |
|(18,[0,1,12],[10.0,1.0,1.0])|0      |0.0       |
|(18,[0,1,4],[10.0,1.0,1.0]) |0      |0.0       |
+----------------------------+-------+----------+
only showing top 5 rows



**Model Evaluation**

Now let's evaluate our model! Let's get the accuracy of our model

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
predDF.columns

['sex',
 'age',
 'province',
 'state',
 'is_dead',
 'is_male',
 'province_index',
 'is_male_OHE',
 'province_index_OHE',
 'features',
 'rawPrediction',
 'probability',
 'prediction']

In [ ]:
evaluator=MulticlassClassificationEvaluator(predictionCol="prediction", labelCol='is_dead', metricName="accuracy")
acc = evaluator.evaluate(predDF)
 
print("Prediction Accuracy: ", acc)

Prediction Accuracy:  0.9733333333333334


In [ ]:
evaluator=MulticlassClassificationEvaluator(predictionCol="prediction", labelCol='is_dead', metricName="f1")
f1Score = evaluator.evaluate(predDF)
 
print("Prediction f1-score: ", f1Score)

Prediction f1-score:  0.9601801801801801


Excellent! Now let's generate the confusion matrix of our predictions

*Hint: we can use `scikit-learn`'s `classification_report`. You will need to transform the predictions into pandas DF first*

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
y_pred=predDF.select("prediction").collect()
y_orig=predDF.select("is_dead").collect()

cm = confusion_matrix(y_orig, y_pred)
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[730   0]
 [ 20   0]]


In [ ]:
#covert to pandas dataframe
pandasDF = predDF.toPandas()
pandasDF.head(5)

,sex,age,province,state,is_dead,is_male,province_index,is_male_OHE,province_index_OHE,features,rawPrediction,probability,prediction
0,female,0.0,Gyeonggi-do,isolated,0,0.0,2.0,(1.0),"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[2.9575684967998965, 0.042431503200103644]","[0.9858561655999655, 0.014143834400034548]",0.0
1,female,0.0,Gyeonggi-do,isolated,0,0.0,2.0,(1.0),"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[2.9575684967998965, 0.042431503200103644]","[0.9858561655999655, 0.014143834400034548]",0.0
2,female,0.0,Gyeongsangbuk-do,released,0,0.0,1.0,(1.0),"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2.9575684967998965, 0.042431503200103644]","[0.9858561655999655, 0.014143834400034548]",0.0
3,female,10.0,Chungcheongbuk-do,released,0,0.0,10.0,(1.0),"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(10.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[2.9575684967998965, 0.042431503200103644]","[0.9858561655999655, 0.014143834400034548]",0.0
4,female,10.0,Gyeonggi-do,isolated,0,0.0,2.0,(1.0),"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(10.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,...","[2.9575684967998965, 0.042431503200103644]","[0.9858561655999655, 0.014143834400034548]",0.0


In [ ]:
#classification report
from sklearn.metrics import classification_report

y_true = pandasDF['is_dead']
y_pred = pandasDF['prediction']

print(classification_report(y_true, y_pred, target_names=['dead', 'survived']))

              precision    recall  f1-score   support

        dead       0.97      1.00      0.99       730
    survived       0.00      0.00      0.00        20

    accuracy                           0.97       750
   macro avg       0.49      0.50      0.49       750
weighted avg       0.95      0.97      0.96       750



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Great Job!

**If you followed the instructions correctly, you should get a total accuracy of 89%, and F1 scores of 92% and 85%**

**Do you think you can improve this accuracy? Let's see what you can do :)**

___
If you have any questions you can reach out to me:

### Omar Hammad
#### Software Engineer
##### Email: ommar365@gmail.com
##### Phone: 01144070145
##### Linkedin: https://www.linkedin.com/in/omar-a-hammad